### import libraries

In [1]:
import pandas as pd 
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.__version__

'2.4.0'

## Part 1 - Data preprocessing

### Preprocessing the training set

In [3]:
data_gen = ImageDataGenerator(
                rescale = 1/255,
                rotation_range=0.4,
                width_shift_range=0.3,
                height_shift_range=0.3,
                shear_range = 0.2,
                zoom_range = 0.2,
                validation_split=0.2,
                horizontal_flip = True)
train_set = data_gen.flow_from_directory(
                            'C:/Users/ARCHITA/Desktop/Machine Learning/0. Projects/facemask detector/dataset',
                            target_size = (256,256),
                            batch_size = 32,
                            class_mode = 'binary',
                            subset='training'
                            )

Found 3274 images belonging to 2 classes.


### Preprocessing the test set

In [4]:
test_set = data_gen.flow_from_directory(
                            'C:/Users/ARCHITA/Desktop/Machine Learning/0. Projects/facemask detector/dataset',
                            target_size = (256,256),
                            batch_size = 32,
                            class_mode = 'binary',
                            subset='validation')

Found 818 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [5]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [6]:
cnn.add(tf.keras.layers.Conv2D(filters = 64 , kernel_size = 3, activation = 'relu', input_shape = [256,256,3]))

### Step 2 - Pooling

In [7]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2 , strides = 2))

### Adding a second convolution layer

In [8]:
cnn.add(tf.keras.layers.Conv2D(filters = 128 , kernel_size = 3, activation = 'relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2 , strides = 2)) 

### Step 3 - Flattening

In [9]:
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dropout(0.2))

### Step 4 - full connection

In [10]:
cnn.add(tf.keras.layers.Dense(units = 128 , activation = 'relu'))

### Step 5 - output layer

In [11]:
cnn.add(tf.keras.layers.Dense(units = 1 , activation = 'sigmoid'))

## Training the CNN

### compiling the CNN

In [12]:
cnn.compile(optimizer = 'adam' , loss = 'binary_crossentropy' , metrics = ['accuracy'])

cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 254, 254, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 127, 127, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 125, 128)     73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 492032)            0         
_________________________________________________________________
dropout (Dropout)            (None, 492032)            0         
_________________________________________________________________
dense (Dense)                (None, 128)               6

### Training the CNN on the trainingset and evaluating it on the test set 

In [13]:
cnn.fit(x = train_set, validation_data = test_set , epochs = 20)

Epoch 1/20
 59/103 [================>.............] - ETA: 3:37 - loss: 2.2389 - accuracy: 0.6269

C:\Users\ARCHITA\Anaconda3\lib\site-packages\PIL\Image.py:993: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


103/103 [==============================] - 509s 5s/step - loss: 1.6697 - accuracy: 0.6646 - val_loss: 0.4017 - val_accuracy: 0.8802
Epoch 2/20
103/103 [==============================] - 439s 4s/step - loss: 0.5119 - accuracy: 0.7832 - val_loss: 0.3133 - val_accuracy: 0.8900
Epoch 3/20
103/103 [==============================] - 433s 4s/step - loss: 0.4584 - accuracy: 0.8180 - val_loss: 0.2981 - val_accuracy: 0.9071
Epoch 4/20
103/103 [==============================] - 446s 4s/step - loss: 0.4063 - accuracy: 0.8365 - val_loss: 0.3411 - val_accuracy: 0.8729
Epoch 5/20
103/103 [==============================] - 432s 4s/step - loss: 0.3973 - accuracy: 0.8389 - val_loss: 0.3040 - val_accuracy: 0.9132
Epoch 6/20
103/103 [==============================] - 440s 4s/step - loss: 0.3616 - accuracy: 0.8551 - val_loss: 0.2470 - val_accuracy: 0.9205
Epoch 7/20
103/103 [==============================] - 434s 4s/step - loss: 0.4053 - accuracy: 0.8305 - val_loss: 0.2441 - val_accuracy: 0.9144
Epoch 8/20

## Making a single prediction

In [14]:
from keras.preprocessing import image
test_image = image.load_img('C:/Users/ARCHITA/Desktop/Machine Learning/0. Projects/facemask detector/images/pic1.jpg', target_size = (256,256))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image , axis = 0)
result = cnn.predict(test_image)
# print(train_set.class_indices)
score = result[0][0]
print(score)
if score>0.5:
    print("No mask")
else:
    print("mask")

0.0
mask


In [15]:
cnn.save('my_model.h5') 

In [16]:
new_model = tf.keras.models.load_model('my_model.h5')

In [17]:
from keras.preprocessing import image
test_image = image.load_img('C:/Users/ARCHITA/Desktop/Machine Learning/0. Projects/facemask detector/images/pic1.jpg', target_size = (256,256))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image , axis = 0)
result = new_model.predict(test_image)
result

array([[0.]], dtype=float32)